<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="6" color='grey'> <b>
Multimodal - Video
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']


# **1 <font color='orange'>|</font> Einführung**

---



Video spielt eine zentrale Rolle in vielen Anwendungen, sei es in der Inhaltsanalyse, der automatischen Objekterkennung oder der Generierung von Inhalten. Durch moderne KI-Technologien können Videoinhalte effizient verarbeitet und ausgewertet werden. In diesem Abschnitt werden vier wesentliche Anwendungsbereiche der Videoverarbeitung vorgestellt:

- **Video-zu-Text (Video-to-Text, VTT)**: Videoinhalte werden in Text umgewandelt, um visuelle Szenen, Handlungen oder Präsentationen schriftlich festzuhalten. Dies erleichtert die Dokumentation und Weiterverarbeitung von visuellen Inhalten.
- **Videoanalyse**: Durch die Analyse von Bildinhalten, Bewegungen, Objekten und Szenen können relevante Informationen aus Videos extrahiert werden. Diese Methode wird häufig in der Sicherheitsüberwachung, im Marketing oder in der Forschung eingesetzt.
- **Video-Summary**: Wichtige Informationen aus langen Videodateien werden extrahiert und in einer kompakten Form dargestellt. Dies ermöglicht eine schnelle Erfassung der Kernaussage, ohne das gesamte Material ansehen zu müssen.
- **Bild-zu-Video (Image-to-Video, ITV)**: Statische Bilder werden in bewegte Videosequenzen umgewandelt. Diese Technik wird für die Animation von Illustrationen, die Erstellung dynamischer Erklärvideos oder die visuelle Inszenierung von Konzepten genutzt.

Diese Techniken erleichtern die Verarbeitung und Nutzung großer Mengen visueller Inhalte und eröffnen neue Möglichkeiten für die Automatisierung und Analyse von Video-Daten.




# **2 <font color='orange'>|</font> Video-zu-Text (VTT)**

---



Die Umwandlung von Videoinhalten in Text, bekannt als Video-Transkription, ermöglicht die automatische Verschriftlichung von Videoszenen, Vorträgen oder Handlungen. Dies erleichtert die Dokumentation, Archivierung und Weiterverarbeitung von Videoinhalten. Moderne Bilderkennung und Spracherkennungstechnologien nutzen KI-Modelle, um visuelle und auditive Elemente präzise zu erfassen und in lesbaren Text umzuwandeln.

Videotranskriptionen sind in vielen Bereichen von Bedeutung – von der Erstellung von Untertiteln und der Zugänglichkeit für Menschen mit Behinderungen über die Erstellung durchsuchbarer Videoinhalte bis hin zur Analyse von Marktforschungsdaten. Durch den Einsatz automatisierter Systeme kann dieser Prozess erheblich beschleunigt werden, wodurch große Mengen an Videomaterial effizient verarbeitet werden können.

<p><font color='black' size="5">
Version auf Basis Transkript
</font></p>

In [ ]:
import openai
from google.colab import files
from IPython.display import display, Markdown
import os

# Nutzer wählt eine Datei aus und lädt sie hoch
uploaded = files.upload()

# Den ersten Dateinamen extrahieren
video_file_path = list(uploaded.keys())[0]

# Temporären Pfad für die Audio-Extraktion erstellen
audio_path = "extracted_audio.mp3"

# FFmpeg verwenden, um Audio aus dem Video zu extrahieren
!ffmpeg -i "{video_file_path}" -q:a 0 -map a "{audio_path}" -y

# Audiodatei öffnen und Whisper nutzen
with open(audio_path, "rb") as audio_file:
    response = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )

# Markdown-Formatierte Ausgabe
markdown_output = f"""
# 🎥 Transkript des Videos

**📂 Datei:** `{video_file_path}`
**🤖 Modell:** `Whisper-1` (für Audiotranskription)

---

{response.text}
"""

# Markdown in Google Colab anzeigen
display(Markdown(markdown_output))

# Temporäre Audiodatei löschen
# os.remove(audio_path)

<p><font color='black' size="5">
Version auf Basis Bilder
</font></p>

In Anlehnung an: [Quelle](https://cookbook.openai.com/examples/gpt_with_vision_for_video_understanding).

In [ ]:
from IPython.display import display, Image, Markdown
from google.colab import files
import cv2
import base64
import time
from openai import OpenAI
import os
import requests

client = OpenAI()

In [ ]:
# Datei-Upload ermöglichen
uploaded = files.upload()  # Öffnet einen Dialog zum Datei-Upload

# Name der hochgeladenen Datei ermitteln
filename = list(uploaded.keys())[0]
print(f"Datei '{filename}' wurde hochgeladen.")

# Video mit OpenCV öffnen
video = cv2.VideoCapture(filename)

# Frames in base64 umwandeln
base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

# Video-Ressourcen freigeben
video.release()
print(len(base64Frames), "Frames wurden eingelesen.")

In [ ]:
display_handle = display(None, display_id=True)
for img in base64Frames:
    display_handle.update(Image(data=base64.b64decode(img.encode("utf-8"))))
    time.sleep(0.025)

In [ ]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Generate a compelling description that I can upload along with the video. Please answer in German.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
params = {
    "model": "gpt-4o-mini",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 200,
}


# Markdown-Formatierte Ausgabe
markdown_output = f"""
# 🎥 Analyse des Videos auf Basis Frames

**📂 Datei:** `{filename}`
**🤖 Modell:** gpt-4o-mini

---
"""

result = client.chat.completions.create(**params)

display(Markdown(markdown_output))
display(Markdown(result.choices[0].message.content))

# **3 <font color='orange'>|</font> Videoanalyse**

---



Durch die Analyse von Bildinhalten, Objekten, Bewegungen und Szenen können relevante Informationen aus Videos extrahiert werden. Die Fokus in diesem Modul ist die Videoanalyse mit Computer Vision, auch **Visual Analysis** genannt, sie ist eine Methode zur automatischen Erkennung und Klassifizierung von Objekten, Personen und Handlungen in Videosequenzen. Mithilfe maschinellen Sehens (Computer Vision) werden Videos auf verschiedene Elemente und Muster untersucht. Diese Technik wird häufig in der Sicherheitsüberwachung, in der Produktionslinie oder bei der Analyse von Social-Media-Inhalten eingesetzt.

Moderne KI-Modelle analysieren nicht nur einzelne Frames, sondern berücksichtigen auch zeitliche Zusammenhänge, um Bewegungen und Handlungen präziser zu erfassen. Unternehmen nutzen diese Technologie, um Kundenverhalten zu verstehen, Sicherheitsrisiken zu erkennen oder Prozesse zu optimieren.

In [ ]:
import openai
from google.colab import files
from IPython.display import display, Markdown, HTML
import cv2
import os
import numpy as np
import base64
import tempfile

# Video hochladen und Frame-Extraction
uploaded = files.upload()
video_file_path = list(uploaded.keys())[0]

def extract_frames(video_path, num_frames=5):
    """Extrahiert Frames aus einem Video"""
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=int)

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames

# Hilfsfunktion für Base64-Kodierung
def get_base64_encoded(image_path):
    """Konvertiert ein Bild in Base64-String"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Frames extrahieren und speichern
frames = extract_frames(video_file_path)
frame_paths = [f"frame_{i}.jpg" for i in range(len(frames))]
for i, frame in enumerate(frames):
    cv2.imwrite(frame_paths[i], frame)

# Audio extrahieren und transkribieren
audio_path = "extracted_audio.mp3"
!ffmpeg -i "{video_file_path}" -q:a 0 -map a "{audio_path}" -y

with open(audio_path, "rb") as audio_file:
    transcribed_text = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    ).text

# Frame-Analyse mit GPT-4o Vision
frame_descriptions = []
for frame_path in frame_paths:
    base64_img = get_base64_encoded(frame_path)
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Du bist ein Experte für Videoanalyse. Beschreibe kurz und präzise, was auf diesem Frame zu sehen ist."},
            {"role": "user", "content": [
                {"type": "text", "text": "Beschreibe diesen Video-Frame mit einem kurzen Satz:"},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_img}"}}
            ]}
        ],
        max_tokens=150
    )
    frame_descriptions.append(response.choices[0].message.content)

# Gesamtanalyse mit GPT-4o-mini
analysis_prompt = f"""
Analysiere den folgenden Videoinhalt basierend auf dem Transkript und den Frame-Beschreibungen.
Gib eine detaillierte Zusammenfassung dessen, was im Video passiert.

### Transkript:
{transcribed_text}

### Frame-Beschreibungen:
{' | '.join([f"Frame {i+1}: {desc}" for i, desc in enumerate(frame_descriptions)])}
"""

analysis_result = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Du bist ein Experte für Videoanalyse."},
        {"role": "user", "content": analysis_prompt}
    ],
    temperature=0.2
).choices[0].message.content

# Ergebnisse anzeigen
frames_html = "".join([
    f"""
    <div style="display: inline-block; margin: 10px; text-align: center;">
        <img src="data:image/jpeg;base64,{get_base64_encoded(frame_path)}" style="width: 200px; border: 1px solid #ddd;">
        <p style="width: 200px; font-size: 12px;">{frame_descriptions[i]}</p>
    </div>
    """ for i, frame_path in enumerate(frame_paths)
])

# Markdown-Anzeige
display(Markdown(f"""
# 🎥 Videoanalyse
**📂 Datei:** `{video_file_path}`
**🤖 Modelle:** `Whisper-1` (Audio), `GPT-4o` (Frames), `GPT-4o-mini` (Gesamtanalyse)

## 📝 Transkript
{transcribed_text}

## 🖼️ Extrahierte Frames und Beschreibungen
"""))
display(HTML(frames_html))
display(Markdown(f"""
## 🧠 Videoanalyse-Ergebnis
{analysis_result}
"""))

# Aufräumen
for path in frame_paths + [audio_path]:
    os.remove(path)

# **4 <font color='orange'>|</font> Video-Objekterkennung**

---

Die Video-Objekterkennung ist ein wichtiger Bereich der Computer Vision, der sich mit der Identifikation und Klassifikation von Objekten in Videomaterial befasst. Im Gegensatz zur statischen Bilderkennung müssen dabei nicht nur einzelne Frames analysiert, sondern auch zeitliche Zusammenhänge berücksichtigt werden.  

Ziel dieses Kapitels ist es, ein grundlegendes Verständnis für die Herausforderungen und Lösungsansätze der Video-Objekterkennung zu vermitteln und praxisnahe Anwendungen zu demonstrieren. Zum Einsatz kommt **YOLO** (You Only Look Once), ein Echtzeit-Objekterkennungsmodell, das durch seine hohe Geschwindigkeit und Genauigkeit besonders für Anwendungen wie autonome Fahrzeuge, Überwachungssysteme und interaktive KI-Systeme geeignet ist.

In [ ]:
!uv pip install --system -q ultralytics opencv-python

In [ ]:
# Import
import ultralytics
from ultralytics import YOLO
from google.colab import files
import cv2
from IPython.display import HTML
from base64 import b64encode

# YOLO-Modell laden
model = YOLO("yolov8m.pt")

# Video-Datei hochladen
uploaded = files.upload()

# Dateinamen abrufen (das erste hochgeladene Video)
video_path = list(uploaded.keys())[0]

# YOLO-Vorhersage auf dem Video durchführen
# Hier werden Objekte erkannt und das Video mit Bounding Boxes gespeichert
results = model.predict(source=video_path, save=True, conf=0.25, save_txt=True, format='avi')

# Das verarbeitete Video anzeigen
video_path = video_path.replace("mp4", "avi")
processed_video_path = f"{results[0].save_dir}/{video_path}"

# Video herunterladen
print(f"Video mit Objekterkennung: {processed_video_path}")
files.download(processed_video_path)

# **5 <font color='orange'>|</font> Bild-zu-Video (TTV)**

---

Der Schwerpunkt liegt auf der Anwendung von Bild-zu-Video (BTV) Large Language Models (LLMs), die mithilfe fortschrittlicher generativer KI-Technologien statische Bilder in realistische Videosequenzen umwandeln. Im Zentrum stehen Modelle wie Sora und Gen-2, leistungsstarke Systeme, die für die Transformation einzelner Bilder oder Bildfolgen in dynamische, kohärente Videoclips optimiert wurden. Diese Modelle wurden speziell entwickelt, um visuell ansprechende und detailreiche Animationen zu erzeugen, was sie besonders für kreative und professionelle Anwendungen attraktiv macht.

Durch den Einsatz tief lernender Algorithmen ermöglichen diese Modelle eine hochqualitative Videosynthese, die nicht nur Bewegungsabläufe realistisch nachbildet, sondern auch komplexe visuelle Effekte erzeugen kann. In diesem Modul werden die Funktionsweise dieser BTV-Modelle analysiert, ihre praktischen Anwendungsfälle untersucht und ihr Einfluss auf verschiedene Branchen bewertet – von Filmproduktion und Werbung über digitale Bildung bis hin zu virtuellen Simulationen und Produktvisualisierungen.

Führende Modelle in diesem Bereich sind:

- **Sora (OpenAI)** – hochrealistische Videogenerierung mit beeindruckender Kohärenz
- **Gen-2 (Runway)** – vielseitiges Modell für kurze kreative Videos
- **Pika Labs** – benutzerfreundliches Tool für einfache Animationen
- **Stable Video Diffusion** – Open-Source-Ansatz für Videogenerierung
- **Lumiere (Google)** – experimentelles Modell mit Fokus auf Bewegungsdynamik

Jedes dieser Modelle hat eigene Stärken und Einsatzgebiete, von der Erstellung von Marketingmaterial über Bildungsvideos bis hin zu kreativen künstlerischen Projekten. Die Anzahl der Frames, die Auflösung und die visuelle Qualität variieren je nach Modell. Die Nutzung dieser Technologie erfordert meist eine API-Verbindung zu den entsprechenden Diensten, da die lokale Ausführung aufgrund der enormen Rechenanforderungen oft nicht praktikabel ist.

In [ ]:
import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import export_to_video
from PIL import Image
from google.colab import files
from IPython.display import display, HTML, Markdown

# Pipeline laden
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt",
    torch_dtype=torch.float16,
    variant="fp16"
)

# Prüfen, ob eine GPU verfügbar ist
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Verwendetes Gerät: {device}")

# Wenn eine CPU verwendet wird, sollte float32 statt float16 genutzt werden
if device == "cpu":
    pipe = pipe.to(torch.float32)

pipe = pipe.to(device)

# Pfad zu deinem Ausgangsbild und Bild laden
uploaded = files.upload()

# Dateinamen abrufen (das erste hochgeladene Video)
image_path = list(uploaded.keys())[0]
# image_path = "/content/avocado-g168eefcd0_1920.jpg"

# Bild mit PIL laden
image = Image.open(image_path)
image = image.resize((512, 512), Image.LANCZOS)

# Parameter für die Videogenerierung
num_frames = 25  # 25 Frames für 5 Sekunden bei 5 FPS
fps = 5  # Frames pro Sekunde
motion_bucket_id = 180  # Stärke der Bewegung (0-999)


# Video generieren
generator = torch.manual_seed(42)  # Für Reproduzierbarkeit
frames = pipe(
    image,  # PIL-Bild-Objekt anstelle eines Strings
    decode_chunk_size=8,
    generator=generator,
    motion_bucket_id=motion_bucket_id,
    num_frames=num_frames,
    fps=fps
).frames[0]

# Video speichern
output_path = "output_video.mp4"
export_to_video(frames, output_path, fps=fps)
files.download(output_path)

# Ergebnisse zusammenfassen
result = {
    "status": "success",
    "message": "Video wurde erfolgreich generiert",
    "video_url": f"Lokal gespeichert als '{output_path}'"
}

print("📊 Status-Informationen")
print("Status:", result["status"])
print("Nachricht:", result["message"])
print("Video-URL:", result["video_url"])

# 📝 Markdown-Ausgabe mit Informationen zum generierten Video
video_info = f"""
## 🎬 Generiertes Video aus Bild

**🖼️ Ausgangsbild:** {image_path}
**🤖 Modell:** Stable Video Diffusion (img2vid-xt)
**⚙️ Parameter:**
- Frames: {num_frames}
- FPS: {fps}
- Motion Bucket ID: {motion_bucket_id}

"""

# Markdown-Ausgabe anzeigen
display(Markdown(video_info))

# **A <font color='orange'>|</font> Aufgabe**
---

Bereit für eine Herausforderung? In diesen Aufgaben werden Sie die leistungsstarken Vision-Modelle von **MediaPipe** kennenlernen. Mit diesen Modellen können sie u.a. Objekterkennung, Handgesten und Gesichtsausdrücke in Echtzeit analysieren. Entdecken Sie die Potenziale der GenAI. Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

[MediaPipe](https://mediapipe-studio.webapps.google.com/home)

[MediaPipe Lösungsleitfaden](https://ai.google.dev/edge/mediapipe/solutions/guide?hl=de)

<p><font color='black' size="5">
Video-Objekterkennung
</font></p>

* **Aufgabe:**
    * Installieren Sie die erforderlichen Bibliotheken (MediaPipe, OpenCV).
    * Laden Sie ein vortrainiertes Objekterkennungsmodell von MediaPipe herunter.
    * Schreiben Sie ein Python-Skript, das ein Bild oder einen Videostream einliest und die erkannten Objekte mit Begrenzungsrahmen und Labels visualisiert.
    * Experimentieren Sie mit verschiedenen Bildern und Videostreams und beobachten Sie die Ergebnisse.
* **Zusatzaufgabe:** Versuchen Sie die Genauigkeit der Objekterkennung zu verbessern, indem sie verschiedene Modelle oder Parameter ausprobieren.


<p><font color='black' size="5">
Handgestenerkennung
</font></p>

* **Aufgabe:**
    * Nutzen Sie das MediaPipe-Modell für die Handgestenerkennung.
    * Erstellen Sie ein Programm, das die Positionen der Handgelenke und Finger erkennt und visualisiert.
    * Implementieren Sie eine einfache Gestenerkennung, z. B. die Erkennung von "Daumen hoch" oder "Daumen runter".
    * Lassen Sie Ihr Programm auf erkannte Gesten mit einer Ausgabe im Terminal reagieren.
* **Zusatzaufgabe:** Die Teilnehmer können komplexere Gesten implementieren oder das Programm mit einer anderen Anwendung (z. B. einem Präsentationsprogramm) verbinden, um Gesten zur Steuerung zu verwenden.



<p><font color='black' size="5">
Gesichtserkennung und -verfolgung
</font></p>

* **Aufgabe:**
    * Verwenden Sie das MediaPipe-Modell für die Gesichtserkennung und -verfolgung.
    * Erstellen Sie ein Programm, das Gesichter in einem Videostream erkennt und die Gesichtspunkte (z. B. Augen, Nase, Mund) verfolgt.
    * Implementieren Sie eine einfache Gesichtsausdrucksanalyse, z. B. die Erkennung von Lächeln oder Stirnrunzeln.
    * Die Daten die erhoben werden können visuell in Echtzeit dargestellt werden.
* **Zusatzaufgabe:** Sie können versuchen, die Gesichtsausdrucksanalyse zu verbessern oder das Programm mit einer anderen Anwendung (z. B. einem Avatar-Programm) zu verbinden, um Gesichtsausdrücke auf den Avatar zu übertragen.
